In [1]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# 1. Get the JSON data from the URL
url = "https://sleepy-poincare-71343e.netlify.app/tracking/records.json"
response = requests.get(url)
data = response.json()

# 2. Flatten the nested structure and convert to pandas DataFrame
records = []
for item in data:
    record = {
        "latitude": item["la"],
        "longitude": item["ln"],
        "date": item["d"]["date"],
        "collarId": item["d"]["collarId"],
        "serialId": item["d"]["serialId"],
        "positionId": item["d"]["positionId"]
    }
    records.append(record)

df = pd.DataFrame(records)

# Optional: preview the DataFrame
print(df)


       latitude  longitude                  date collarId   serialId  \
0     -2.192725  34.788930  2025-11-12T15:30:40Z    13312  T5HS-5607   
1     -2.293685  34.563048  2025-11-12T15:30:40Z    17324  T5HS-7561   
2     -2.192640  34.788693  2025-11-12T15:30:08Z    13312  T5HS-5607   
3     -2.293333  34.562987  2025-11-12T15:30:08Z    17324  T5HS-7561   
4     -2.205470  34.806368  2025-11-12T09:30:42Z    13312  T5HS-5607   
...         ...        ...                   ...      ...        ...   
12257 -1.907895  34.824855  2025-09-13T01:30:30Z    13288  T5HS-5583   
12258 -1.908048  34.824657  2025-09-13T01:00:30Z    13288  T5HS-5583   
12259 -1.908145  34.824708  2025-09-13T00:30:30Z    13288  T5HS-5583   
12260 -1.907840  34.824533  2025-09-13T00:30:00Z    13288  T5HS-5583   
12261 -1.916422  34.819123  2025-09-13T00:00:30Z    13288  T5HS-5583   

      positionId  
0      150594082  
1      150594084  
2      150594071  
3      150594083  
4      150594081  
...          ...  
12

In [14]:
# len(df)

In [15]:
# # df['serialId'].value_counts()
# len(df['positionId'].unique())

In [16]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options

# # Setup headless Chrome
# options = Options()
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

# url = "https://www.serengeti-tracker.org/track/T5HS-5607"  # page with species
# driver.get(url)

# # Wait for the species element to load
# import time
# time.sleep(5)  # wait for JS to render content

# # Find the element containing species
# species_element = driver.find_element(By.CLASS_NAME, "details")
# print(species_element.text)  

# species_element.text

# driver.quit()


T5HS-5607
SPECIES
Zebra
LAST TRACKED
Sunday 9th Nov 2025 16:30pm


In [ ]:
# Setup headless Chrome
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

for serialid in df['serialId'].unique():

    url = f"https://www.serengeti-tracker.org/track/{serialid}"  # page with species
    try:
        driver.get(url)

        # Wait for the species element to load
        import time
        time.sleep(5)  # wait for JS to render content

        # Find the element containing species
        species_element = driver.find_element(By.CLASS_NAME, "details")
        print(species_element.text)  # should print 'zebra'
    except:
        print(f"SOMETHING WENT WRONG WITH Serial ID {serialid}")
driver.quit()



T5HS-5608
SPECIES
Wildebeest
LAST TRACKED
Monday 10th Nov 2025 10:30am
T5HS-7291
SPECIES
Wildebeest
LAST TRACKED
Monday 10th Nov 2025 10:30am
T5HS-7287
SPECIES
Wildebeest
LAST TRACKED
Monday 10th Nov 2025 10:30am
T5HS-7301
SPECIES
Wildebeest
LAST TRACKED
Monday 10th Nov 2025 10:30am
SOMETHING WENT WRONG WITH Serial IDT5HS-7561
T5HS-7298
SPECIES
Wildebeest
LAST TRACKED
Sunday 9th Nov 2025 22:30pm
T5HS-5601
SPECIES
Wildebeest
LAST TRACKED
Sunday 9th Nov 2025 22:30pm
SOMETHING WENT WRONG WITH Serial IDT5HS-7566
SOMETHING WENT WRONG WITH Serial IDT5H-1861
T5HS-5607
SPECIES
Zebra
LAST TRACKED
Sunday 9th Nov 2025 16:30pm
T5HS-7296
SPECIES
Wildebeest
LAST TRACKED
Sunday 9th Nov 2025 14:30pm
SOMETHING WENT WRONG WITH Serial IDT5HS-7302
T5HS-7306
SPECIES
Wildebeest
LAST TRACKED
Saturday 8th Nov 2025 22:30pm
T5HS-5583
SPECIES
Wildebeest
LAST TRACKED
Saturday 8th Nov 2025 19:30pm
T5HS-5594
SPECIES
Wildebeest
LAST TRACKED
Saturday 8th Nov 2025 14:30pm
T5HS-6666
SPECIES
Eland
LAST TRACKED
Saturday 